In [5]:
import pandas as pd
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from pandas.io.json import json_normalize

## Connect to our service account

In [21]:
scope = ['https://spreadsheets.google.com/feeds']
credentials = ServiceAccountCredentials.from_json_keyfile_name('Blend_Gsheet_and_Jupyter-2bf5ca30cd89.json', scope)
gc = gspread.authorize(credentials)


## Get candiadate data sheet from Google Drive 

In [22]:
spreadsheet_key = '1f0OwtmuTk1fTdhnn4tuvVcPCZjdb00D79dWw4RFhYs0'
book = gc.open_by_key(spreadsheet_key)
worksheet = book.worksheet("Candidate Data")
table = worksheet.get_all_values()
table[:8]

[['Order',
  'Candidate',
  'Position',
  'Start Date',
  'End Date',
  'Years of Experience',
  'Points',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  ''],
 ['45',
  'Donald Trump',
  'President',
  '1/20/2017',
  '1/20/2021',
  '4.002739726',
  '104',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  ''],
 ['45',
  'Donald Trump',
  'Secretary of State',
  '',
  '',
  '0',
  '0',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  ''],
 ['45',
  'Donald Trump',
  'Secretary of Defense',
  '',
  '',
  '0',
  '0',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  ''],
 ['45',
  'Donald Trump',
  'Secretary of Treasury',
  '',
  '',
  '0',
  '0',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  ''],
 ['45',
  'Donald Trump',
  'Vice President',
  '',
  '',
  '0',
  '0',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  ''],
 ['45',
  'Donald Trump',
  'Attorney General',
  '',

## Convert sheet data into pandas dataframe

In [52]:
##Convert table data into a DataFrame
df = pd.DataFrame(table[1:], columns=table[0])

##Only keep columns we need
df = df[['Order', 'Candidate', 'Position', 'Start Date', 'End Date', 'Years of Experience', 'Points']]

##Convert number strings to float and ints
df = df.apply(pd.to_numeric, errors='ignore')

##Convert data strings to datetime format
df['End Date'] = pd.to_datetime(df['End Date'], infer_datetime_format=True)
df['Start Date'] = pd.to_datetime(df['Start Date'], infer_datetime_format=True)

df.head()
# df.dtypes

,Order,Candidate,Position,Start Date,End Date,Years of Experience,Points
0,45,Donald Trump,President,2017-01-20,2021-01-20,4.00274,104.0
1,45,Donald Trump,Secretary of State,NaT,NaT,0.00000,0.0
2,45,Donald Trump,Secretary of Defense,NaT,NaT,0.00000,0.0
3,45,Donald Trump,Secretary of Treasury,NaT,NaT,0.00000,0.0
4,45,Donald Trump,Vice President,NaT,NaT,0.00000,0.0


## And walla! You have converted your Google Sheet data into a nice, clean pandas dataframe. If you change the data in the sheet and want to update the dataframe, simply rerun all the code from:

book = gc.open_by_key(spreadsheet_key) 

## Push from Jupyter notebook to Google Sheet

In [50]:
#Do some manipulation of the data
candidate_groups = df.groupby('Candidate')
scores_df = candidate_groups.sum()
scores_df['Order'] = candidate_groups.first()
scores_df.head()

,Order,Years of Experience,Points
Candidate,,,
Abraham Lincoln,16,6.336986,114.00
Andrew Jackson,7,19.884932,246.85
Andrew Johnson,17,25.443836,204.45
Barack Obama,44,12.054795,208.55
Benjamin Harrison,23,12.887671,137.70


In [55]:
from df2gspread import df2gspread as d2g
wks_name = 'Jupyter Manipulated Data'
d2g.upload(scores_df, spreadsheet_key, wks_name, credentials=credentials, row_names=True)

AttributeError: 'Worksheet' object has no attribute 'get_int_addr'